# 0. Preprocessing

#### Starting with an XDD Query, we obtain PDF ids and eventually the PDFs themselves. We can then run COSMOS to extract outputs like this:
    

In [9]:
with open("documents_sidarthe--COSMOS-data.json", "r") as f:
    text = f.read()
    print(text.replace('\n', ' ')[:500])

[     {         "pdf_name": "41591_2020_Article_883.pdf",         "dataset_id": "documents_5Feb",         "page_num": 1,         "img_pth": "/mytmp/images/41591_2020_Article_883.pdf_1_pad",         "pdf_dims": [             0.0,             0.0,             612.0,             792.0         ],         "bounding_box": [             103.0,             51.0,             1341.0,             212.0         ],         "classes": [             "Section Header",             "Page Footer",             "Fig


#### We can consolidate just the "content" fields to get just the text of the paper

In [10]:
# We have a straightforward script for this but let's look at the output directly
with open("sidarthe.txt", "r") as f:
    text = f.read()
    print(text.replace('\n', ' ')[:500])

 Modelling the COVID-19 epidemic and implementation of population-wide Modelling the COVID-19 epidemic and implementation of population-wide interventions in Italy Giulia Giordano 1 ✉, Franco Blanchini2, Raffaele Bruno3,4, Patrizio Colaneri5,6, Alessandro Di Filippo3, Angela Di Matteo3 and Marta Colaneri3 In Italy, 128,948 confirmed cases and 15,887 deaths of peo- discharged due to recovery and 15,887 had died7. In the early days of ple  who  tested  positive  for  SARS-CoV-2  were  registered  


#### From the COSMOS output, we can also keep metadata like the paper name and doi for later

In [ ]:
import json
with open("sidarthe_info.json", "r") as f:
    info = json.load(f)
    info_s = json.dumps(info)
    print(info_s)

# 1. Extracting variables and annotating them

#### Using our API (powered by GPT-3), we can extract variables from the paper alongside a list of possible definitions, and ground each of these variables to the MIRA DKG. The JSON format of our output can be found [here](https://github.com/mikecafarella/mitaskem/blob/main/JSONformat.md)

In [ ]:
import requests
import ast
API_ROOT = "http://100.26.10.46/"
GPT_KEY = ""

In [ ]:
with open("sidarthe_short.txt", "r") as f:
        text = f.read()           
        dct = {"text":text, "gpt_key":GPT_KEY}           
        r = requests.post(API_ROOT + "annotation/find_text_vars/", params=dct)
        print(r)

In [ ]:
json_str = r.text
ast.literal_eval(json_str)

In [ ]:
with open("jan_demo_intermediate.json", "w+") as f:
    f.write(json_str)

# 2. Extracting LaTeX from formula images

#### As we demoed last week, we can extract latex from formulas (powered by `pix2tex`), also through a public API. Here is a formula image from the SIDARTHE paper:

In [ ]:
from IPython import display
display.Image("sidarthe-eqn1.png")

In [ ]:
import os
directory = '../../resources/images/SIDARTHE'
latex_strs = []
 
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    latex_str = !python3 img_latex.py -p {f}
    print(latex_str)
    latex_strs.append(latex_str[0])

#### Clearly, this is not perfect, but it is workable

# 3. Linking variables from the LaTeX formulas to variables from the text

#### Again, the output JSON format can be found [here](https://github.com/mikecafarella/mitaskem/blob/main/JSONformat.md)

In [ ]:
full_json_str = json_str
latex_strs = latex_strs 
for latex_str in latex_strs:
    print(latex_str)
    dct2 = {"json_str":full_json_str, "formula": latex_str, "gpt_key":GPT_KEY}           

    r2 = requests.post(API_ROOT + "annotation/link_latex_to_vars/", params=dct2)
    print(r2)
    full_json_str = r2.text

In [ ]:
ast.literal_eval(full_json_str)

In [ ]:
with open("jan_demo_full.json", "w+") as f:
    f.write(full_json_str)

# 4. Getting a Petri net (as a pyascet) from code 

#### We have the python function that describes the SIDARTHE dynamics:

In [ ]:
with open("../../resources/jan_evaluation/scenario_2_sidarthe/sidarthe_code.py", "r") as f:
    code = f.read()
print(code)

#### We can use the (new, correctly named version of the) MIT APIs presented last week to get Petri net components from this graph

In [ ]:
dict_petri = {"code": code, "gpt_key": GPT_KEY}
places = requests.post(API_ROOT + "petri/get_places", params=dict_petri).text
print(places)

In [ ]:
transitions = requests.post(API_ROOT + "petri/get_transitions", params=dict_petri).text
print(transitions)

In [ ]:
arcs = requests.post(API_ROOT + "petri/get_arcs", params=dict_petri).text
print(arcs)

#### We can then convert these outputs into a py-acset

In [ ]:
dict_acset = {"places_str": places, "transitions_str": transitions, "arcs_str": arcs}

acset = requests.post(API_ROOT + "petri/get_pyacset", params=dict_acset).text

pyacset_s = acset
print(acset)

In [ ]:
ast.literal_eval(pyacset_s)

# 5. Linking the annotations to the py-acset and paper info

#### Finally, we bring everything together: the annotations from earlier, the pyacset and the info from the paper from the very beginning

In [ ]:
dct3 = {"pyacset_str":pyacset_s, "annotations_str":full_json_str, "info_str":info_s}           
r3 = requests.post(API_ROOT + "annotation/link_annos_to_pyacset/", params=dct3)
print(r3)

In [ ]:
ast.literal_eval(r3.text)

#### Data in this format is ingestible by TA4!

In [44]:
with open("jan_demo_full.json", "w+") as f:
    f.write(full_json_str)

# 4. Getting a Petri net (as a pyascet) from code 

#### We have the python function that describes the SIDARTHE dynamics:

In [66]:
with open("../../resources/jan_evaluation/scenario_2_sidarthe/sidarthe_code.py", "r") as f:
    code = f.read()
print(code)

# define SIDARTHE_model
def SIDARTHE_model(y, t, alpha, beta, gamma, delta, epsilon, mu, zeta, lamda, eta, rho, theta, kappa, nu, xi, sigma, tau):
    S, I, D, A, R, T, H, E = y
    dSdt = -S*(alpha(t)*I + beta(t)*D + gamma(t)*A + delta(t)*R)
    dIdt = S*(alpha(t)*I + beta(t)*D + gamma(t)*A + delta(t)*R) - (epsilon(t) + zeta(t) + lamda(t))*I
    dDdt = epsilon(t)*I - (eta(t) + rho(t))*D
    dAdt = zeta(t)*I - (theta(t) + mu(t) + kappa(t))*A
    dRdt = eta(t)*D + theta(t)*A - (nu(t) + xi(t))*R
    dTdt = mu(t)*A + nu(t)*R - (sigma(t) + tau(t))*T
    dHdt = lamda(t)*I + rho(t)*D + kappa(t)*A + xi(t)*R + sigma(t)*T
    dEdt = tau(t)*T
    
    return dSdt, dIdt, dDdt, dAdt, dRdt, dTdt, dHdt, dEdt


#### We can use the (new, correctly named version of the) MIT APIs presented last week to get Petri net components from this graph

In [67]:
dict_petri = {"code": code, "gpt_key": GPT_KEY}
places = requests.post(API_ROOT + "petri/get_places", params=dict_petri).text
print(places)

["S"," I"," D"," A"," R"," T"," H"," E"]


In [68]:
transitions = requests.post(API_ROOT + "petri/get_transitions", params=dict_petri).text
print(transitions)

["alpha"," beta"," gamma"," delta"," epsilon"," mu"," zeta"," lamda"," eta"," rho"," theta"," kappa"," nu"," xi"," sigma"," tau"]


In [69]:
arcs = requests.post(API_ROOT + "petri/get_arcs", params=dict_petri).text
print(arcs)

[["S"," I"],["I"," D"],["I"," A"],["I"," R"],["D"," E"],["A"," R"],["A"," T"],["R"," H"],["T"," H"],["I"," H"],["D"," H"],["A"," H"],["R"," H"],["T"," E"]]


#### We can then convert these outputs into a py-acset

In [91]:
dict_acset = {"places_str": places, "transitions_str": transitions, "arcs_str": arcs}

acset = requests.post(API_ROOT + "petri/get_pyacset", params=dict_acset).text

pyacset_s = acset
print(acset)

{"S":[{"sname":"S","uid":1},{"sname":"I","uid":2},{"sname":"D","uid":3},{"sname":"A","uid":4},{"sname":"R","uid":5},{"sname":"T","uid":6},{"sname":"H","uid":7},{"sname":"E","uid":8}],"T":[{"tname":"alpha","uid":10},{"tname":" beta","uid":11},{"tname":" gamma","uid":12},{"tname":" delta","uid":13},{"tname":" epsilon","uid":14},{"tname":" mu","uid":15},{"tname":" zeta","uid":16},{"tname":" lamda","uid":17},{"tname":" eta","uid":18},{"tname":" rho","uid":19},{"tname":" theta","uid":20},{"tname":" kappa","uid":21},{"tname":" nu","uid":22},{"tname":" xi","uid":23},{"tname":" sigma","uid":24},{"tname":" tau","uid":25}],"I":[{"it":1,"is":1},{"it":2,"is":2},{"it":3,"is":2},{"it":4,"is":2},{"it":5,"is":3},{"it":6,"is":4},{"it":7,"is":4},{"it":8,"is":5},{"it":9,"is":6},{"it":10,"is":2},{"it":11,"is":3},{"it":12,"is":4},{"it":13,"is":5},{"it":14,"is":6}],"O":[{"ot":1,"os":2},{"ot":2,"os":3},{"ot":3,"os":4},{"ot":4,"os":5},{"ot":5,"os":8},{"ot":6,"os":5},{"ot":7,"os":6},{"ot":8,"os":7},{"ot":9,"os

In [92]:
ast.literal_eval(pyacset_s)

{'S': [{'sname': 'S', 'uid': 1},
  {'sname': 'I', 'uid': 2},
  {'sname': 'D', 'uid': 3},
  {'sname': 'A', 'uid': 4},
  {'sname': 'R', 'uid': 5},
  {'sname': 'T', 'uid': 6},
  {'sname': 'H', 'uid': 7},
  {'sname': 'E', 'uid': 8}],
 'T': [{'tname': 'alpha', 'uid': 10},
  {'tname': ' beta', 'uid': 11},
  {'tname': ' gamma', 'uid': 12},
  {'tname': ' delta', 'uid': 13},
  {'tname': ' epsilon', 'uid': 14},
  {'tname': ' mu', 'uid': 15},
  {'tname': ' zeta', 'uid': 16},
  {'tname': ' lamda', 'uid': 17},
  {'tname': ' eta', 'uid': 18},
  {'tname': ' rho', 'uid': 19},
  {'tname': ' theta', 'uid': 20},
  {'tname': ' kappa', 'uid': 21},
  {'tname': ' nu', 'uid': 22},
  {'tname': ' xi', 'uid': 23},
  {'tname': ' sigma', 'uid': 24},
  {'tname': ' tau', 'uid': 25}],
 'I': [{'it': 1, 'is': 1},
  {'it': 2, 'is': 2},
  {'it': 3, 'is': 2},
  {'it': 4, 'is': 2},
  {'it': 5, 'is': 3},
  {'it': 6, 'is': 4},
  {'it': 7, 'is': 4},
  {'it': 8, 'is': 5},
  {'it': 9, 'is': 6},
  {'it': 10, 'is': 2},
  {'it': 1

# 5. Linking the annotations to the py-acset and paper info

#### Finally, we bring everything together: the annotations from earlier, the pyacset and the info from the paper from the very beginning

In [93]:
dct3 = {"pyacset_str":pyacset_s, "annotations_str":full_json_str, "info_str":info_s}           
r3 = requests.post(API_ROOT + "annotation/link_annos_to_pyacset/", params=dct3)
print(r3)

<Response [200]>


In [94]:
ast.literal_eval(r3.text)

{'1': {'type': 'variable',
  'name': 'S',
  'id': 'v0',
  'text_annotations': [' Susceptible (uninfected)'],
  'dkg_annotations': [['ncit:C171133', 'COVID-19 Infection'],
   ['ido:0000514', 'susceptible population']],
  'equation_annotations': {'\\delta(t)=-S(t)(a l(t)+\\theta D(t)+r A(t)+\\delta t(t)': ['S']},
  'file': '41591_2020_Article_883.pdf',
  'doi': 'https://doi.org/10.1038/s41591-020-0883-7'},
 '2': {'type': 'variable',
  'name': 'I',
  'id': 'v1',
  'text_annotations': [' Infected (asymptomatic or pauci-symptomatic infected, undetected)'],
  'dkg_annotations': [['ido:0000511', 'infected population'],
   ['ncit:C171133', 'COVID-19 Infection']],
  'equation_annotations': {'{\\dot{A}}(t)=\\zeta I(t)-(\\theta+\\mu+\\kappa)A(t)': ['I'],
   '{\\hat{D}}(t)=\\varepsilon I(t)-(\\eta+\\rho)D(t)': ['I']},
  'file': '41591_2020_Article_883.pdf',
  'doi': 'https://doi.org/10.1038/s41591-020-0883-7'},
 '3': {'type': 'variable',
  'name': 'D',
  'id': 'v2',
  'text_annotations': [' Diagno

#### Data in this format is ingestible by TA4!